Пример сделан на основе блога https://proglib.io/p/moem-dataset-rukovodstvo-po-ochistke-dannyh-v-python-2020-03-27

### 1. Загрузка данных

In [ ]:
# импорт библиотек
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib
plt.style.use('ggplot')
from matplotlib.pyplot import figure

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12,8)
pd.options.mode.chained_assignment = None #отключаем предупреждения для нагляднсоти вывода
# чтение данных
df = pd.read_csv('train.csv')

### 2. Разведочный анализ загруженных данных 

In [ ]:
# размеры массивов и типы данных
print(df.shape)
print(df.dtypes)

In [ ]:
# числовые данные
df_numeric = df.select_dtypes(include=[np.number])
numeric_cols = df_numeric.columns.values
print(numeric_cols)

In [ ]:
# категорийные данные
df_non_numeric = df.select_dtypes(exclude=[np.number])
non_numeric_cols = df_non_numeric.columns.values
print(non_numeric_cols)

### 2.1 Поиск пропущенных значений

In [ ]:
df.count()

In [ ]:
df.isna().sum()

### Тепловая карта с пропущенными значениями

In [ ]:
cols = df.columns[:30] # первые 30 колонок
# определяем цвета 
# желтый - пропущенные данные, синий - не пропущенные
colours = ['#000099', '#ffff00'] 
sns.heatmap(df[cols].isnull(), cmap=sns.color_palette(colours))

### Пропущенные данные в процентах

In [ ]:
for col in df.columns:
    pct_missing = np.mean(df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

In [ ]:
# Ещё одна функция проверки пропущенных значений
def missing_values_table(df):
        # Общее количество отсутствующих значений
        mis_val = df.isnull().sum()
        
        # Процент пропущенных значений
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Составление таблицы с результатами
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Перименование столбцов
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Сортировка таблицы по проценту отсутствующих значений по убыванию
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Вывод результатов
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns
missing_values_table(df)

### Гистограмма пропущенных значений

In [ ]:
for col in df.columns:
    missing = df[col].isnull()
    num_missing = np.sum(missing)
    
    if num_missing > 0:  
        print('created missing indicator for: {}'.format(col))
        df['{}_ismissing'.format(col)] = missing


# затем на основе индикатора строим гистограмму
ismissing_cols = [col for col in df.columns if 'ismissing' in col]
df['num_missing'] = df[ismissing_cols].sum(axis=1)

df['num_missing'].value_counts().reset_index().sort_values(by='index').plot.bar(x='index', y='num_missing')

In [ ]:
df

### 2.2 Борьба с пропущенными значениями

#### Способ 1. Удаление всех строк с пропущенными значениями

In [ ]:
# отбрасываем строки с большим количеством пропусков
ind_missing = df[df['num_missing'] > 35].index
df_less_missing_rows = df.drop(ind_missing, axis=0)

#### Способ 2. Удаление всех признаков, у которых много пропусков в строках

In [ ]:
# у признака hospital_beds_raion имеет высокий процент недостающих значений – 47%. 
# отказываемся от этого признака
cols_to_drop = ['hospital_beds_raion']
df_less_hos_beds_raion = df.drop(cols_to_drop, axis=1)

#### Способ 3. Самостоятельное заполнение пропусков (нулями, средним, модой, медианой или специальным значеним)

In [ ]:
#пропущенные значения признака life_sq заменяем на медианное значение
med = df['life_sq'].median()
print(med)
df['life_sq'] = df['life_sq'].fillna(med)

In [ ]:
#применяем для всех числовых данных
df_numeric = df.select_dtypes(include=[np.number])
numeric_cols = df_numeric.columns.values

for col in numeric_cols:
    missing = df[col].isnull()
    num_missing = np.sum(missing)
    
    if num_missing > 0:  # only do the imputation for the columns that have missing values.
        print('imputing missing values for: {}'.format(col))
        df['{}_ismissing'.format(col)] = missing
        med = df[col].median()
        df[col] = df[col].fillna(med)

In [ ]:
#применяем для всех категориальных данных
df_non_numeric = df.select_dtypes(exclude=[np.number])
non_numeric_cols = df_non_numeric.columns.values

for col in non_numeric_cols:
    missing = df[col].isnull()
    num_missing = np.sum(missing)
    
    if num_missing > 0:  # only do the imputation for the columns that have missing values.
        print('imputing missing values for: {}'.format(col))
        df['{}_ismissing'.format(col)] = missing
        
        top = df[col].describe()['top'] # impute with the most frequent value.
        df[col] = df[col].fillna(top)

In [ ]:
# А можем заменить на какие-то свои значения
# категориальные признаки
df['sub_area'] = df['sub_area'].fillna('_MISSING_')
# численные признаки
df['life_sq'] = df['life_sq'].fillna(-999)

### 2.3 Обнаружение выбросов

In [ ]:
df['life_sq'].hist(bins=100)

In [ ]:
df.boxplot(column=['life_sq'])

In [ ]:
df = df.loc[df['life_sq']<1000] #отрезаем выбросы

In [ ]:
df.boxplot(column=['life_sq'])

Задание.
1. Загрузить исходный датасет.
2. Провести очистку и трансформацию исходного датасета исходя из следующих условий: удалить признаки, с более чем 25% пропусками в значениях. Оставшиеся пропуски заполнить медианными значениями.
3. Построить матрицу корреляции Для признаков `timestamp`, `full_sq`, `life_sq`, `floor`, `build_year`, `num_room`, `price_doc`. Сделать вывод о корреляционной зависимости признаков.
4. Провести анализ наличия выбросов для не менее трёх признаков из `timestamp`, `full_sq`, `life_sq`, `floor`, `build_year`, `num_room`. Удалить строки с выбросами у данных признаков.
5. Стандартизировать числовые данные (`sklearn.preprocessing.StandartScaler()`) и преобразовать категориальные данные (`sklearn.preprocessing.LabelEncoder()`).
6. Уменьшить размерность признаков на основе метода главных компонент (`sklearn.decomposition.PCA`).
7. Построить две регрессионные модели на основе случайного леса (`sklearn.ensemble.RandomForestRegressor`) и метода опорных векторов (`sklearn.svm.SVR`). Путём поиска по сетке подобрать наилучшие параметры для них.
8. Провести сравнение моделей на тестовой выборке `test.csv`.